In [65]:
import pandas as pd
#pip install --upgrade google-api-python-client
#pip install pip install google-cloud-bigquery
from google.cloud import bigquery
# pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
from  bq_helper import BigQueryHelper
import os
import numpy as np
from scipy.spatial import KDTree

In [66]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="//Users/abhishekgawande/Downloads/OneDrive_4_11-16-2021/fall-21-329000-5fc2cb5ab9a1.json"


bq_assist=BigQueryHelper("bigquery-public-data","epa_historical_air_quality")



In [27]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [28]:
bq_assist.list_tables()


['air_quality_annual_summary',
 'co_daily_summary',
 'co_hourly_summary',
 'hap_daily_summary',
 'hap_hourly_summary',
 'lead_daily_summary',
 'no2_daily_summary',
 'no2_hourly_summary',
 'nonoxnoy_daily_summary',
 'nonoxnoy_hourly_summary',
 'o3_daily_summary',
 'o3_hourly_summary',
 'pm10_daily_summary',
 'pm10_hourly_summary',
 'pm25_frm_daily_summary',
 'pm25_frm_hourly_summary',
 'pm25_nonfrm_daily_summary',
 'pm25_nonfrm_hourly_summary',
 'pm25_speciation_daily_summary',
 'pm25_speciation_hourly_summary',
 'pressure_daily_summary',
 'pressure_hourly_summary',
 'rh_and_dp_daily_summary',
 'rh_and_dp_hourly_summary',
 'so2_daily_summary',
 'so2_hourly_summary',
 'temperature_daily_summary',
 'temperature_hourly_summary',
 'voc_daily_summary',
 'voc_hourly_summary',
 'wind_daily_summary',
 'wind_hourly_summary']

In [29]:
# Abhishek: 17 Nov 2021

query="""
   SELECT * from `bigquery-public-data.epa_historical_air_quality.pressure_daily_summary`
   LIMIT 5
    """

bq_df=bq_assist.query_to_pandas(query)
# print(bq_df.head())
print(bq_df.columns)

Index(['state_code', 'county_code', 'site_num', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter_name', 'sample_duration',
       'pollutant_standard', 'date_local', 'units_of_measure', 'event_type',
       'observation_count', 'observation_percent', 'arithmetic_mean',
       'first_max_value', 'first_max_hour', 'aqi', 'method_code',
       'method_name', 'local_site_name', 'address', 'state_name',
       'county_name', 'city_name', 'cbsa_name', 'date_of_last_change'],
      dtype='object')


In [21]:
query="""



select date_local,
         cast(state_code as int) as state_code ,
         state_name,
         cast(county_code as int) as county_code,
         county_name,
         site_num,
         address,
         latitude,
         longitude,
         from `bigquery-public-data.epa_historical_air_quality.co_daily_summary`
         where date_local>='2010-01-01' AND aqi>=0
        
    """

co_daily_df=bq_assist.query_to_pandas(query)
print(co_daily_df.head())

   date_local  state_code state_name  county_code county_name site_num  \
0  2012-11-01           2     Alaska           20  Anchorage      0052   
1  2012-06-09          17   Illinois          143      Peoria     0036   
2  2012-06-16          17   Illinois          143      Peoria     0036   
3  2012-10-28          17   Illinois          143      Peoria     0036   
4  2012-10-27          40   Oklahoma          143       Tulsa     1127   

              address   latitude   longitude  
0        727 L Street  61.215027 -149.903111  
1  1005 N. UNIVERSITY  40.700072  -89.613414  
2  1005 N. UNIVERSITY  40.700072  -89.613414  
3  1005 N. UNIVERSITY  40.700072  -89.613414  
4  3520 1/2 N. PEORIA  36.204902  -95.976537  


In [119]:
all_pairs_co = co_daily_df[["state_code","county_code","latitude", "longitude"]]
all_pairs_co = all_pairs_co.groupby(["state_code", "county_code"]).agg(np.mean)
all_pairs_co

latitude   longitude
state_code county_code                       
1          73           33.527446  -86.849886
2          20           61.220724 -149.824742
           90           64.845523 -147.726645
4          12           34.243900 -113.558600
           13           33.450166 -112.094069
...                           ...         ...
72         61           18.422595  -66.120012
           113          18.009558  -66.627249
           127          18.449818  -66.054227
80         2            32.532256 -116.616162
           6            31.714251 -106.440162

[238 rows x 2 columns]

In [120]:
all_pairs_temp = pd.read_csv("daily_TEMP_2021.csv")
all_pairs_temp = all_pairs_temp[["State Code", "County Code", "Latitude", "Longitude"]]
all_pairs_temp = all_pairs_temp.groupby(["State Code", "County Code"]).agg(np.mean)
all_pairs_temp

Latitude   Longitude
State Code County Code                       
1          53           31.092100  -87.543500
           73           33.537242  -86.829556
2          68           63.723200 -148.967600
4          3            32.009410 -109.389060
           5            36.058640 -112.183600
...                           ...         ...
56         5            44.652200 -105.290300
           35           42.429930 -109.829679
           37           41.763060 -108.734440
           39           44.203083 -110.610215
80         2            32.632532 -115.490334

[209 rows x 2 columns]

In [121]:
state_county_pairs = pd.read_csv("state_county_pairs.csv")
state_county_pairs

,state_code,county_code
0,1,73
1,4,13
2,4,19
3,5,119
4,6,1
...,...,...
95,49,49
96,50,7
97,50,21
98,51,13


In [122]:
# all_pairs_co

In [123]:
coordinates = state_county_pairs.merge(all_pairs_co, how = "left" , on = ["state_code", "county_code"])
coordinates

,state_code,county_code,latitude,longitude
0,1,73,33.527446,-86.849886
1,4,13,33.450166,-112.094069
2,4,19,32.210307,-110.898958
3,5,119,34.756189,-92.281296
4,6,1,37.783330,-122.219348
...,...,...,...,...
95,49,49,40.269842,-111.672404
96,50,7,44.499713,-73.056629
97,50,21,43.608056,-72.982778
98,51,13,38.857700,-77.059220


In [124]:
coordinates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   state_code   100 non-null    int64  
 1   county_code  100 non-null    int64  
 2   latitude     100 non-null    float64
 3   longitude    100 non-null    float64
dtypes: float64(2), int64(2)
memory usage: 3.9 KB


In [125]:
coordinates.to_csv("coordinates.csv", index = False)

In [126]:
# Finding k nearest coordinates

In [127]:
# from scipy.spatial import KDTree

# display(coordinates.head())

# pts = coordinates.to_numpy()

# pts_nn = np.hstack((pts, np.stack(KDTree(pts).query(pts, k=3))[:,:,1].T))


In [128]:
# Finding k nearest coordinates

In [129]:
# display(pts_nn)

In [130]:
import scipy

mat = scipy.spatial.distance.cdist(coordinates[['latitude','longitude']], 
                              coordinates[['latitude','longitude']], metric='euclidean')
mat.shape

(100, 100)

In [176]:
new_df = pd.DataFrame(mat)

In [177]:
new_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,25.244301,24.085114,5.568664,35.624589,35.542112,35.436243,33.146743,38.031144,28.674554,...,10.035194,19.676545,9.245847,10.391263,10.447035,25.721923,17.625113,17.143961,11.147590,10.095261
1,25.244301,0.000000,1.722075,19.855772,11.013518,11.611531,10.893519,8.426472,14.127274,3.495574,...,15.246978,5.884838,17.205746,15.130466,14.847608,6.832699,40.571101,40.408858,35.449712,25.520559
2,24.085114,1.722075,0.000000,18.790925,12.617837,13.294325,12.511173,10.036004,15.797397,4.642578,...,14.052077,4.469819,15.800965,13.839476,13.643211,8.096562,39.787829,39.592239,34.486457,24.959879
3,5.568664,19.855772,18.790925,0.000000,30.090705,29.980493,29.898533,27.629734,32.465694,23.320664,...,4.971337,14.475845,5.838263,5.706810,5.387234,20.159748,21.552821,21.231777,15.764961,9.447735
4,35.624589,11.013518,12.617837,30.090705,0.000000,2.013136,0.281063,2.587068,3.561465,8.401251,...,25.840368,16.871767,28.100942,25.885019,25.464881,10.836086,49.619379,49.579908,45.172906,34.075439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,25.721923,6.832699,8.096562,20.159748,10.836086,10.181498,10.587875,8.881699,12.525599,8.468038,...,16.580202,9.962281,19.467379,16.954590,16.270323,0.000000,38.846749,38.833373,34.641978,23.271834
96,17.625113,40.571101,39.787829,21.552821,49.619379,49.014176,49.384206,47.414242,51.269394,44.059614,...,26.514530,35.730833,26.679540,27.234383,26.926426,38.846749,0.000000,0.894710,6.917590,15.598764
97,17.143961,40.408858,39.592239,21.231777,49.579908,49.009634,49.347554,47.350220,51.285650,43.901244,...,26.201301,35.493007,26.259938,26.891439,26.615488,38.833373,0.894710,0.000000,6.259653,15.638977
98,11.147590,35.449712,34.486457,15.764961,45.172906,44.791042,44.956486,42.828733,47.167552,38.940218,...,20.700933,30.228725,20.378049,21.268896,21.120159,34.641978,6.917590,6.259653,0.000000,12.446476


In [178]:
sorted_new_df = np.argsort(new_df)
sorted_new_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,40,39,88,89,87,47,45,48,49,...,20,4,23,13,19,11,24,80,8,41
1,1,2,9,61,18,15,17,66,14,91,...,33,31,97,96,54,86,53,50,51,41
2,2,1,91,9,66,61,18,15,17,14,...,33,31,97,96,54,86,53,50,51,41
3,3,89,79,59,49,90,78,94,0,45,...,20,4,23,13,19,11,24,80,8,41
4,4,19,6,20,23,11,13,22,24,16,...,32,33,97,96,31,54,86,53,50,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,61,42,29,27,26,1,30,28,66,...,33,97,96,31,54,86,53,50,51,41
96,96,97,54,32,31,50,53,86,33,67,...,80,23,4,13,20,19,11,24,8,41
97,97,96,54,32,31,53,33,86,50,67,...,23,4,13,20,80,19,11,24,8,41
98,98,34,52,63,85,82,84,72,81,65,...,20,4,23,13,19,11,24,80,8,41


In [179]:
# sorted_new_df.iloc[:,1:6]

In [180]:
nearest_5_locations = sorted_new_df.iloc[:,1:6]

In [181]:
nearest_5_locations.columns

RangeIndex(start=1, stop=6, step=1)

In [182]:
for i in nearest_5_locations.columns:
    coordinates["neighbor" + str(i)] = nearest_5_locations.iloc[:,i-1]


In [183]:
coordinates_with_5_nearest_neighbors = coordinates.copy()

In [184]:
coordinates_with_5_nearest_neighbors

,state_code,county_code,latitude,longitude,neighbor1,neighbor2,neighbor3,neighbor4,neighbor5
0,1,73,33.527446,-86.849886,40,39,88,89,87
1,4,13,33.450166,-112.094069,2,9,61,18,15
2,4,19,32.210307,-110.898958,1,91,9,66,61
3,5,119,34.756189,-92.281296,89,79,59,49,90
4,6,1,37.783330,-122.219348,19,6,20,23,11
...,...,...,...,...,...,...,...,...,...
95,49,49,40.269842,-111.672404,61,42,29,27,26
96,50,7,44.499713,-73.056629,97,54,32,31,50
97,50,21,43.608056,-72.982778,96,54,32,31,53
98,51,13,38.857700,-77.059220,34,52,63,85,82


In [185]:
state_county_pairs

,state_code,county_code
0,1,73
1,4,13
2,4,19
3,5,119
4,6,1
...,...,...
95,49,49
96,50,7
97,50,21
98,51,13


In [187]:
state_county_pairs.loc[coordinates_with_5_nearest_neighbors["neighbor1"],:].reset_index(drop= True)

,state_code,county_code
0,13,121
1,4,19
2,4,13
3,47,157
4,6,75
...,...,...
95,32,3
96,50,21
97,50,7
98,11,1


In [188]:
coordinates_with_5_nearest_neighbors[['neighbor1_state_code','neighbor1_county_code']] = state_county_pairs.loc[coordinates_with_5_nearest_neighbors["neighbor1"],:].reset_index(drop= True)



In [189]:
coordinates_with_5_nearest_neighbors

,state_code,county_code,latitude,longitude,neighbor1,neighbor2,neighbor3,neighbor4,neighbor5,neighbor1_state_code,neighbor1_county_code
0,1,73,33.527446,-86.849886,40,39,88,89,87,13,121
1,4,13,33.450166,-112.094069,2,9,61,18,15,4,19
2,4,19,32.210307,-110.898958,1,91,9,66,61,4,13
3,5,119,34.756189,-92.281296,89,79,59,49,90,47,157
4,6,1,37.783330,-122.219348,19,6,20,23,11,6,75
...,...,...,...,...,...,...,...,...,...,...,...
95,49,49,40.269842,-111.672404,61,42,29,27,26,32,3
96,50,7,44.499713,-73.056629,97,54,32,31,50,50,21
97,50,21,43.608056,-72.982778,96,54,32,31,53,50,7
98,51,13,38.857700,-77.059220,34,52,63,85,82,11,1


In [190]:
for i in range(1,6):
    coordinates_with_5_nearest_neighbors[['neighbor'+str(i)+'_state_code','neighbor'+str(i)+'_county_code']] = state_county_pairs.loc[coordinates_with_5_nearest_neighbors["neighbor"+str(i)],:].reset_index(drop= True)


In [191]:
coordinates_with_5_nearest_neighbors

,state_code,county_code,latitude,longitude,neighbor1,neighbor2,neighbor3,neighbor4,neighbor5,neighbor1_state_code,neighbor1_county_code,neighbor2_state_code,neighbor2_county_code,neighbor3_state_code,neighbor3_county_code,neighbor4_state_code,neighbor4_county_code,neighbor5_state_code,neighbor5_county_code
0,1,73,33.527446,-86.849886,40,39,88,89,87,13,121,13,89,47,37,47,157,47,9
1,4,13,33.450166,-112.094069,2,9,61,18,15,4,19,6,25,32,3,6,73,6,65
2,4,19,32.210307,-110.898958,1,91,9,66,61,4,13,48,141,6,25,35,1,32,3
3,5,119,34.756189,-92.281296,89,79,59,49,90,47,157,40,143,29,510,22,33,48,113
4,6,1,37.783330,-122.219348,19,6,20,23,11,6,75,6,13,6,81,6,95,6,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,49,49,40.269842,-111.672404,61,42,29,27,26,32,3,16,1,8,69,8,31,8,1
96,50,7,44.499713,-73.056629,97,54,32,31,50,50,21,25,27,9,5,9,3,23,5
97,50,21,43.608056,-72.982778,96,54,32,31,53,50,7,25,27,9,5,9,3,25,25
98,51,13,38.857700,-77.059220,34,52,63,85,82,11,1,24,5,34,7,42,101,42,21


In [196]:
# df.drop(['B', 'C'], axis=1)
neighbors = coordinates_with_5_nearest_neighbors[["state_code","county_code", "neighbor1_state_code","neighbor1_county_code",\
                                                 "neighbor2_state_code","neighbor2_county_code","neighbor3_state_code",\
                                                  "neighbor3_county_code","neighbor4_state_code","neighbor4_county_code",\
                                                  "neighbor5_state_code", "neighbor5_county_code"
                                                 ]]

In [197]:
neighbors

,state_code,county_code,neighbor1_state_code,neighbor1_county_code,neighbor2_state_code,neighbor2_county_code,neighbor3_state_code,neighbor3_county_code,neighbor4_state_code,neighbor4_county_code,neighbor5_state_code,neighbor5_county_code
0,1,73,13,121,13,89,47,37,47,157,47,9
1,4,13,4,19,6,25,32,3,6,73,6,65
2,4,19,4,13,48,141,6,25,35,1,32,3
3,5,119,47,157,40,143,29,510,22,33,48,113
4,6,1,6,75,6,13,6,81,6,95,6,41
...,...,...,...,...,...,...,...,...,...,...,...,...
95,49,49,32,3,16,1,8,69,8,31,8,1
96,50,7,50,21,25,27,9,5,9,3,23,5
97,50,21,50,7,25,27,9,5,9,3,25,25
98,51,13,11,1,24,5,34,7,42,101,42,21


In [198]:
neighbors.to_csv("neighbors.csv",index=False)